# Opacity Machine

---

## Notes

- Run `create_atom_data.ipynb` before using this notebook.
- Baseline model is generated using [Andreas's script](https://github.com/epassaro/tardis-setups/blob/carsus-paper/rad_trans_models/run_model_toy06.py) with `kurucz_cd23_latest.h5` atom data, pickled with `protocol=0`.

In [1]:
import os
import pickle
import pandas as pd
import astropy.units as u
from tardis.io.atom_data import AtomData
from tardis.io.config_reader import Configuration
from tardis.simulation import Simulation

/home/epassaro/miniconda3/envs/tardis/lib/python3.6/importlib/_bootstrap.py:219: QAWarning: pyne.data is not yet QA compliant.
  return f(*args, **kwds)


## Set variables

In [2]:
EPOCH = '20d'
TARDIS_CONFIG = 'blondin_model_compare_06.yml'
ATOM_DATA_DIR = os.path.expanduser('~/Downloads/tardis-data')
PICKLE_DIR = '/storage/epassaro/Toy_06/kurucz'
OUTPUT_DIR = 'output'
NTHREADS = 32

In [3]:
SAVEPATH = os.path.join(OUTPUT_DIR, EPOCH)
os.makedirs(SAVEPATH, exist_ok=True)

In [4]:
!ls $PICKLE_DIR

toy06_t10.0_v17000.0.pickle  toy06_t20.0_v5500.0.pickle
toy06_t15.0_v10000.0.pickle  toy06_t5.0_v20500.0.pickle


In [5]:
!ls $ATOM_DATA_DIR

kurucz_cd23_latest_chianti_Ca_1.h5
kurucz_cd23_latest_chianti_Fe_1.h5
kurucz_cd23_latest_chianti_Fe_2.h5
kurucz_cd23_latest_chianti_Ni_1.h5
kurucz_cd23_latest_chianti_S_0.h5
kurucz_cd23_latest_chianti_S_1.h5
kurucz_cd23_latest_chianti_S_2.h5
kurucz_cd23_latest_chianti_Si_1-2_S_0-2_Ca_1_Fe_1-2_Ni_1.h5
kurucz_cd23_latest_chianti_Si_1.h5
kurucz_cd23_latest_chianti_Si_2.h5
kurucz_cd23_latest.h5


In [6]:
def save(sim, fname):
    """ Docstring """
    with pd.HDFStore(fname) as hdf:
        hdf.put('wavelength', 
                pd.Series(sim.runner.spectrum.wavelength.value))
        hdf.put('luminosity_density_lambda', 
                pd.Series(sim.runner.spectrum_integrated.luminosity_density_lambda.value))
        hdf.put('t_rad', pd.Series(sim.model.t_rad.value))
        hdf.put('w', pd.Series(sim.model.w))
        hdf.put('abundance', 
                sim.model.abundance)
    return

In [7]:
VELOCITIES = {'5d': 20500.0, '10d': 17000., '15d': 10000., '20d': 5500.}

## Baseline model

In [8]:
PICKLED_SIM = os.path.join(PICKLE_DIR, 
                           'toy06_t{}_v{}.pickle'.format(float(EPOCH.rstrip('d')), VELOCITIES[EPOCH]))

In [9]:
# Pickle load
with open(PICKLED_SIM, 'rb') as pm:
    baseline_simulation = pickle.load(pm)

baseline_model = baseline_simulation.model

# Quick save
dump = os.path.join(SAVEPATH, 'kurucz_cd23_latest_{}.h5'.format(EPOCH))
save(baseline_simulation, dump)

## Run the opacity machine

In [10]:
fnames = [ fname for fname in os.listdir(ATOM_DATA_DIR) if fname.endswith('.h5') ]
fnames = [ fname for fname in fnames if fname != 'kurucz_cd23_latest.h5' ]

for fname in fnames:
    tardis_config = Configuration.from_yaml(TARDIS_CONFIG)
    tardis_config['montecarlo']['nthreads'] = NTHREADS
    tardis_config['montecarlo']['iterations'] = 1
    atom_data = AtomData.from_hdf(os.path.join(ATOM_DATA_DIR, fname))   
    simulation = Simulation.from_config(tardis_config, atom_data=atom_data, 
                                        model=baseline_model)
    simulation.run()
    
    # Check if opacity machine is working as expected
    assert len(simulation.iterations_t_inner.value) == 1
    assert baseline_simulation.iterations_t_inner.value[-1] == simulation.iterations_t_inner.value
    assert (simulation.model.t_rad.value == baseline_simulation.model.t_rad.value).all()
    assert (simulation.model.w == baseline_simulation.model.w).all()
    
    # Quick save
    dump = os.path.join(SAVEPATH, '{}_{}.h5'.format(fname.strip('.h5'), EPOCH))
    save(simulation, dump)

[py.warnings         ][WARNING]  /home/epassaro/miniconda3/envs/tardis/lib/python3.6/site-packages/IPython/core/async_helpers.py:68: PerformanceWarning: indexing past lexsort depth may impact performance.
  coro.send(None)
 (warnings.py:99)
[tardis.io.atom_data.base][INFO   ]  Read Atom Data with UUID=5f16aba8c2c811eabf65f8f21e712851 and MD5=b07453657e551a17fda5adc13c349341. (base.py:187)
[tardis.io.atom_data.base][INFO   ]  Non provided atomic data: collision_data, collision_data_temperatures, synpp_refs, photoionization_data (base.py:193)
[py.warnings         ][WARNING]  /home/epassaro/miniconda3/envs/tardis/lib/python3.6/site-packages/tardis_sn-3.0.dev3477-py3.6-linux-x86_64.egg/tardis/plasma/properties/atomic.py:229: FutureWarning: .labels was deprecated in version 0.24.0. Use .codes instead.
  zeta_data["atomic_number"] = zeta_data.index.labels[0] + 1
 (warnings.py:99)
[py.warnings         ][WARNING]  /home/epassaro/miniconda3/envs/tardis/lib/python3.6/site-packages/tardis_sn-3.0.

[tardis.simulation.base][INFO   ]  Simulation finished in 1 iterations and took 16.06 s (base.py:380)
[tardis.io.atom_data.base][INFO   ]  Read Atom Data with UUID=20eb92c0c2c911eaa9f4f8f21e712851 and MD5=27eafd1c7d87dfe6569bf263f9039dcc. (base.py:187)
[tardis.io.atom_data.base][INFO   ]  Non provided atomic data: collision_data, collision_data_temperatures, synpp_refs, photoionization_data (base.py:193)
[py.warnings         ][WARNING]  /home/epassaro/miniconda3/envs/tardis/lib/python3.6/site-packages/tardis_sn-3.0.dev3477-py3.6-linux-x86_64.egg/tardis/plasma/properties/atomic.py:229: FutureWarning: .labels was deprecated in version 0.24.0. Use .codes instead.
  zeta_data["atomic_number"] = zeta_data.index.labels[0] + 1
 (warnings.py:99)
[py.warnings         ][WARNING]  /home/epassaro/miniconda3/envs/tardis/lib/python3.6/site-packages/tardis_sn-3.0.dev3477-py3.6-linux-x86_64.egg/tardis/plasma/properties/atomic.py:230: FutureWarning: .labels was deprecated in version 0.24.0. Use .codes 

[tardis.simulation.base][INFO   ]  Luminosity emitted = 1.00155e+43 erg / s Luminosity absorbed = 2.38486e+43 erg / s Luminosity requested = 1.05928e+43 erg / s (base.py:447)
[tardis.simulation.base][INFO   ]  Simulation finished in 1 iterations and took 16.05 s (base.py:380)
[py.warnings         ][WARNING]  /home/epassaro/miniconda3/envs/tardis/lib/python3.6/site-packages/IPython/core/async_helpers.py:68: PerformanceWarning: indexing past lexsort depth may impact performance.
  coro.send(None)
 (warnings.py:99)
[tardis.io.atom_data.base][INFO   ]  Read Atom Data with UUID=24e4e778c2c911eabfa7f8f21e712851 and MD5=86b1e89e8e19074f1a44e176b87673d8. (base.py:187)
[tardis.io.atom_data.base][INFO   ]  Non provided atomic data: collision_data, collision_data_temperatures, synpp_refs, photoionization_data (base.py:193)
[py.warnings         ][WARNING]  /home/epassaro/miniconda3/envs/tardis/lib/python3.6/site-packages/tardis_sn-3.0.dev3477-py3.6-linux-x86_64.egg/tardis/plasma/properties/atomic.

[tardis.plasma.properties.atomic][WARNING]  Zeta_data missing - replaced with 1s. Missing ions: [(14, 15), (16, 17), (20, 21), (26, 27), (27, 28), (28, 29)] (atomic.py:251)
[tardis.simulation.base][INFO   ]  Starting iteration 1/1 (base.py:325)
[tardis.simulation.base][INFO   ]  Luminosity emitted = 1.00719e+43 erg / s Luminosity absorbed = 2.37857e+43 erg / s Luminosity requested = 1.05928e+43 erg / s (base.py:447)
[tardis.simulation.base][INFO   ]  Simulation finished in 1 iterations and took 13.69 s (base.py:380)
[py.warnings         ][WARNING]  /home/epassaro/miniconda3/envs/tardis/lib/python3.6/site-packages/IPython/core/async_helpers.py:68: PerformanceWarning: indexing past lexsort depth may impact performance.
  coro.send(None)
 (warnings.py:99)
[tardis.io.atom_data.base][INFO   ]  Read Atom Data with UUID=637bb594c2c811ea98adf8f21e712851 and MD5=a88bafaaac6827e4239e0e967792babe. (base.py:187)
[tardis.io.atom_data.base][INFO   ]  Non provided atomic data: collision_data, collisi